# 👟 Behaviors

In [ ]:
if __name__ == "__main__":
    %pip install -q -r requirements.txt

In [ ]:
import ipywidgets as W
import traitlets as T

In [ ]:
with __import__("importnb").Notebook():
    from _index import make_a_simple_example

```{hint}
After rendering the cell below, select _Create New View For Output_ from the output's right-click menu to see more things added
```

In [ ]:
fg = make_a_simple_example()
fg

## `NodeSelection`

The `NodeSelection` behavior allows for selecting one or more nodes from the browser, or
updating from the kernel.

In [ ]:
from ipyforcegraph.behaviors import NodeSelection

selection = NodeSelection()
fg.behaviors = [selection]

```{hint}
Note that the nodes changed colors. Click a node to select it, or use <kbd>ctrl</kbd> or <kbd>shift</kbd> to select multiple nodes.
```

The selection is handed back from the client, and can be used with other widgets.

In [ ]:
ui_selection = W.TagsInput(
    placeholder="select some nodes",
    description="selected",
    allowed_tags=sorted(fg.source.nodes.id),
)
T.link((selection, "selected"), (ui_selection, "value"))
ui_selection

## `NodeLabels`

Node labels can be revealed when hovering over the node. By default the node's `id`
column will be used.

In [ ]:
from ipyforcegraph.behaviors import NodeLabels

node_labels = NodeLabels()

fg.behaviors = [selection, node_labels]